In [1]:
import numpy as np
from random import randint, shuffle, random

In [2]:
# Chargement de la matrice adjacence à partir d'un fichier
def load_matrix():
    mat = []
    with open('mat_adjacence', 'r') as file:   
        for l in file.readlines():  
            val = [int(x) for x in l.split()]
            mat.append(val)            
    return mat

# Créer une matrice d'adjacence dont le cout du circuit Hamiltonien optimal est celui de la périphérie  
def create_matrix(n_ville):
    mat = np.zeros((n_ville, n_ville))
    
    for i in range(n_ville):
        for j in range(i, n_ville):
            cout = randint(2, 10)
            if i == j:
                cout = 0
            i_after = i+1 if i+1 < n_ville else 0
            if i_after == j:
                cout = 1
                 
            mat[i][j] = cout
            mat[j][i] = cout
    mat[n_ville-1][0] = 1
    mat[0][n_ville-1] = 1

    return mat
 

In [3]:
#mat=[[0,2,1],[2,0,1],[1,2,0]]
#population=[]


# Création de la population initial, de taille size
def population_init(mat, size, n_lives=5):
    population = []
    
    for _ in range(0, size):
        circuit = list(range(0, len(mat[0])))
        shuffle(circuit) # circuit random
        population.append([circuit, n_lives])
        
    return population

#population_init(mat, 5)

In [4]:
#mat = [ [0,5,2], [4,0,3], [1,5,0] ]

def fitness(path, cost_mat): # Renvoie le coût d'un chemin, par rapport à la matrice d'adjacence
    travel_length = 0
    
    for i in range(0, len(path)):
        depart = path[i-1] # On compte aussi l'arrète entre la premiere et dernière ville (-1 en python -> dernier élément) 
        arrival = path[i]
        travel_length += cost_mat[depart][arrival]
    
    return travel_length  

#fitness([1,0,2], mat)

In [5]:
def mort(pop):
    # Décremente la vie d'un individu et tue si <= 0
    new_pop = []
    
    for path in pop:
        path[1] -= 1
        
        # On ne garde que les individus vivants
        if path[1] >= 0:
            new_pop.append(path)
            
    return new_pop

In [6]:
def selection(population, mat_cost): 
    sum = 0
    # Somme des fitness pour ponderation
    for indiv in population:
        sum += fitness(indiv[0], mat_cost) 
    
    # Selection des individus de la population suivant un random
    new_pop = []
    for indiv in population:
        f = fitness(indiv[0], mat_cost)
        # Plus le fitness est bas, plus l'individu a de chance d'être selectionné
        if random() * sum > f:
            new_pop.append(indiv)
            
    return new_pop
    
        


In [7]:
def croisement(population, nb_enfants, life_time):
    for _ in range(0, nb_enfants // 2):
        # Selection des parents
        indice_parent1 = randint(0, len(population)-1)
        indice_parent2 = randint(0, len(population)-1)
        #print("indice du parent1:",indice_parent1,"  indice du parent 2:",indice_parent2)
        #print("taille de la pop:",len(population)-1)

        parent1=population[indice_parent1][0]
        parent2=population[indice_parent2][0]
        indice_same=randint(1, len(parent1) - 1)
        enfant1 = []
        enfant2 = []
        enfant1_temp = []
        enfant2_temp = []

        for x in range(0, indice_same):
            enfant1_temp.append(parent1[x])
            enfant2_temp.append(parent2[x])

        for y in range(indice_same, len(parent1)):
            enfant1_temp.append(parent2[y])
            enfant2_temp.append(parent1[y])

        for a in range(0, len(enfant1_temp)):
            if enfant1_temp[a] not in enfant1:
                enfant1.append(enfant1_temp[a])
            if enfant2_temp[a] not in enfant2:
                enfant2.append(enfant2_temp[a])
        
        for b in range(0, len(parent1)):
            if b not in enfant1:
                enfant1.append(b)
            if b not in enfant2:
                enfant2.append(b)
        
        population.append([enfant1, life_time])
        population.append([enfant2, life_time])
        #print(population)

#pop=[[[0,1,2,3],5],[[1,2,3,0],5]]
#croisement_v2(pop)
#print(pop)

def select_indiv(population, quantity): # Choisit deux chemins de parents  
    i = 1
    selection = []
    while i <= quantity:
        # On selectionne un individu dans la population
        r_index = randint(0, len(population)-i)
        indiv = population[r_index][0]
        # On sépare le parent du reste des individus pour le choix du second parent, en le metant à la fin de la liste.
        population[r_index], population[-i] = population[-i], population[r_index]
        selection.append(indiv)
        i += 1  
    return selection
"""
def change(path, val, index): # Change une valeur de la liste, et rectifie les redondances
    old_val = path[index] 
    path[index] = val
    ##### Rectification #####
    # On parcours en suivant les valeurs redondants
    while old_val not in path:
        # Cherche l'indice de l'element redondant
        i = path.index(val)
        if i == index: # On cherche l'autre indice
            i = path.index(val, i+1)
        # Sauvegarde pour la prochaine itération
        val = old_val
        path[i], old_val = old_val, path[i]
        index = i

def find_missing(arr):
    n = len(arr)
    missing = [] 
    for node in range(n):
        if node not in arr:
            missing.append(node)
    return missing 

def pick_index(arr): # tire un indice aléatoire de la liste  
    return randint(0, len(arr)-1)
 
def change(path, modif_array, index): # Change suivant un array avec index
    
    if len(modif_array) + index > len(path):
        raise Exception("Taille de tableau dépassé") 
    size = len(modif_array)
    # On remplace les anciennes valeurs par les nouvelle 
    left = path[:index]
    right = path[index + size:] 
    mod_arr = list(modif_array)
    missing = find_missing(left+mod_arr+right)
    # Elimination des redondances en esquivant la partie de la liste changé
    # On check chaque éléments rajouté et on tire parmis les elements manquants
    while missing: # Tant qu'il y a des elements manquants 
        added_node = mod_arr.pop() # On cherche parmis les elements ajouté s'il n'y a pas une ville redondante
        if added_node in left:
            i = left.index(added_node)   
            left[i] = missing.pop(pick_index(missing)) # Ajout d'un element manquant  
        if added_node in right:
            i = right.index(added_node)   
            right[i] = missing.pop(pick_index(missing)) # Ajout d'un element manquant 

    return left + modif_array + right
     

def croisement(population, nb_enfants, life_time, change_size=5): # Génère une liste d'enfants
    n_ville = len(population[0][0]) 
    enfants = []
    # Choix des parents (retourne une liste des chemins des parents selectionnés)
    parents = select_indiv(population, nb_enfants) 
    
    for i in range(0, nb_enfants, 2): 
        ###### Croisement entre les deux parents ######
        parent_1 = parents[i]
        parent_2 = parents[i+1]
        # On créer deux enfants
        enfant_1 = list(parent_1)
        enfant_2 = list(parent_2) 
        
        # On change une des valeurs du premier enfant avec une valeur du second parent.
        r_index = randint(0, n_ville-1-change_size)
        enfant_1 = change(enfant_1, parent_2[r_index:r_index+change_size], r_index)
        enfant_2 = change(enfant_2, parent_1[r_index:r_index+change_size], r_index)  
        
        enfants.append([enfant_1, life_time])
        enfants.append([enfant_2, life_time])
    
    return enfants
"""
 

'\ndef change(path, val, index): # Change une valeur de la liste, et rectifie les redondances\n    old_val = path[index] \n    path[index] = val\n    ##### Rectification #####\n    # On parcours en suivant les valeurs redondants\n    while old_val not in path:\n        # Cherche l\'indice de l\'element redondant\n        i = path.index(val)\n        if i == index: # On cherche l\'autre indice\n            i = path.index(val, i+1)\n        # Sauvegarde pour la prochaine itération\n        val = old_val\n        path[i], old_val = old_val, path[i]\n        index = i\n\ndef find_missing(arr):\n    n = len(arr)\n    missing = [] \n    for node in range(n):\n        if node not in arr:\n            missing.append(node)\n    return missing \n\ndef pick_index(arr): # tire un indice aléatoire de la liste  \n    return randint(0, len(arr)-1)\n \ndef change(path, modif_array, index): # Change suivant un array avec index\n    \n    if len(modif_array) + index > len(path):\n        raise Exception

In [8]:
def mutation(pop, mutation_amount, mutation_influence):  # Quantité de mutation et influence en pourcentage
    last_index = len(pop) - 1
    n_ville = len(pop[0][0])
    stop_index = len(pop) - mutation_amount * len(pop)
    # Tant qu'il reste des mutations à faire
    while last_index >= stop_index :
        # Random entre seulement les parties de la population qui n'ont pas déjà recu de mutation
        i = randint(0, last_index) 
        # On applique la mutation
        for _ in range(int(mutation_influence * n_ville)):
            curr_indiv = pop[i][0]
            # Indices random
            r_index_1 = randint(0, n_ville-1) 
            r_index_2 = randint(0, n_ville-1)
            while r_index_2 == r_index_1: # forcer une autre valeur
                r_index_2 = randint(0, n_ville-1)
            # Swap de deux valeurs randoms
            curr_indiv[r_index_1], curr_indiv[r_index_2] = curr_indiv[r_index_2], curr_indiv[r_index_1]
            
        # On deplace l'individu muté vers la partie de la population déjà muté
        pop[i], pop[last_index] = pop[last_index], pop[i]
        # On decremente l'indice de séparation des deux parties du tableau
        last_index -= 1

In [9]:
def verif(population, mat_cost):
    valeur = float('inf') 
    for indiv in population:
        f = fitness(indiv[0], mat_cost)
        if f <= valeur:
            valeur = f
            chemin = indiv[0]
    return chemin, valeur
#mat=[[0,3,4,2],[5,0,6,1],[4,6,0,3],[2,1,3,0]]
#pop=[[[0,1,2,3],5],[[3,1,2,0],5],[[0,3,1,2],5],[[3,2,1,0],5]]
#val,chemin=verif(pop,999)
#print(val,"  ",chemin)

In [10]:
def algo_genetique(mat_cost, verbal=False):  
    ###### Paramètres ######
    time_max = 100 # Nombre de cycle max
    population_size = 10 # Taille initial de la population
    life_time = 10 # Nombre de cycle avant la mort d'un individu
    nb_enfants = 2 # Nombre d'enfant par cycle
    # Pourcentage de mutation
    mutation_amount = 0.2 # Taux d'individu a muter dans la population
    mutation_influence = 0.2 # Taux de changement sur l'individu à muter
    
    if verbal:
        print(mat_cost)
    
    time = 0 # Cycle actuel
    val_min = float("inf")
    population = population_init(mat_cost, population_size, life_time) 
    population = selection(population, mat_cost)
     
    while time < time_max and len(population) >= 2: 
        #croisement(population, nb_enfants, life_time)
        
        # On insert les enfants dans la populations
        croisement(population, nb_enfants, life_time) #population += 
        mutation(population, mutation_amount, mutation_influence)
        population = selection(population, mat_cost)
        population = mort(population) 
        time += 1
        
        if verbal:
            chemin_min, val_min = verif(population, mat_cost) 
            print("iteration:", time, ", nb individu:", len(population), "val_min :", val_min ) 
    
    chemin_min, val_min = verif(population, mat_cost) 
    if verbal: 
        print("Chemin min: ", chemin_min, " pour un cout de: ", val_min)
    return chemin_min, val_min
    
algo_genetique(create_matrix(10), verbal=True)

[[ 0.  1.  2.  5.  9.  7.  9.  9.  6.  1.]
 [ 1.  0.  1.  9.  7.  6.  3.  9.  3.  2.]
 [ 2.  1.  0.  1.  3.  2.  9.  9.  4.  6.]
 [ 5.  9.  1.  0.  1.  3.  3.  3.  8. 10.]
 [ 9.  7.  3.  1.  0.  1.  3.  9.  6.  3.]
 [ 7.  6.  2.  3.  1.  0.  1.  4.  7.  9.]
 [ 9.  3.  9.  3.  3.  1.  0.  1.  6.  6.]
 [ 9.  9.  9.  3.  9.  4.  1.  0.  1.  4.]
 [ 6.  3.  4.  8.  6.  7.  6.  1.  0.  1.]
 [ 1.  2.  6. 10.  3.  9.  6.  4.  1.  0.]]
iteration: 1 , nb individu: 11 val_min : 31.0
iteration: 2 , nb individu: 12 val_min : 31.0
iteration: 3 , nb individu: 13 val_min : 31.0
iteration: 4 , nb individu: 15 val_min : 31.0
iteration: 5 , nb individu: 17 val_min : 31.0
iteration: 6 , nb individu: 18 val_min : 31.0
iteration: 7 , nb individu: 18 val_min : 31.0
iteration: 8 , nb individu: 19 val_min : 25.0
iteration: 9 , nb individu: 20 val_min : 25.0
iteration: 10 , nb individu: 21 val_min : 25.0
iteration: 11 , nb individu: 16 val_min : 25.0
iteration: 12 , nb individu: 17 val_min : 25.0
iteration: 13 

([7, 5, 6, 8, 4, 3, 1, 2, 0, 9], 35.0)